In [30]:
from onvif import ONVIFCamera

In [31]:
# !which python

In [32]:
PATH_TO_WSDL = '/Library/Frameworks/Python.framework/Versions/2.7/wsdl'

In [33]:
# url = '192.168.15.43'
# port = 80
# mycam = ONVIFCamera(url, port, 'ilyahandsup', 'xp3BJtSrR1IZ', PATH_TO_WSDL)

url = '192.168.15.42'
port = 80
mycam = ONVIFCamera(url, port, 'ilyahandsup', 'xp3BJtSrR1IZ', PATH_TO_WSDL)

# url = '192.168.15.42'
# port = 80
# mycam = ONVIFCamera(url, port, 'admin', 'Supervisor', PATH_TO_WSDL)

# url = '192.168.15.42'
# port = 80
# mycam = ONVIFCamera(url, port, 'admin', 'Supervisor', PATH_TO_WSDL)

# url = '192.168.15.52'
# port = 2000
# mycam = ONVIFCamera(url, port, 'admin', 'Supervisor', PATH_TO_WSDL)

resp = mycam.devicemgmt.GetHostname()
print 'My camera`s hostname: ' + str(resp.Name)

My camera`s hostname: Hikvision


In [39]:
# ptz = mycam.create_ptz_service()
media = mycam.create_media_service()
media_profile = media.GetProfiles()[0]
imaging = mycam.create_imaging_service()

# mycam.devicemgmt.GetDeviceInformation() # not working on 42
# ptz.GetStatus()
# media_profile
# imaging

In [40]:
# req = ptz.create_type('GetStatus')
# req.ProfileToken = media_profile._token
# ptz.GetStatus(req)

In [41]:
def req(name = None):
    res = {}
    if name != None:
        res = ptz.create_type(name)
    res['ProfileToken'] = media_profile._token
    return res

In [47]:
# moveCT([1,1,0], [1,2,3])
# move(-1, -1, -1, 2)
# req('ContinuousMove')
# ptz.GetServiceCapabilities(req('GetServiceCapabilities'))
req.Focus.Continuous.Speed = 1
imaging.Move(req)
# req

In [56]:
from onvif import ONVIFCamera
import time
PATH_TO_WSDL = '/Library/Frameworks/Python.framework/Versions/2.7/wsdl'


class Camera():
    def __init__(self, url, port, login, password):
        self.cam = ONVIFCamera(url, port, login, password, PATH_TO_WSDL)
        self.media = self.cam.create_media_service()
        self.ptz = self.cam.create_ptz_service()
        self.imaging = self.cam.create_imaging_service()
        
        self.media_profile = self.media.GetProfiles()[0]
        self.media_token = self.media_profile._token
        self.video_token = self.media_profile.VideoSourceConfiguration.SourceToken
        
        self.possibilities = {}
        self.possibilities['Absolute'] = self.ptz.GetNodes()[0].SupportedPTZSpaces.AbsolutePanTiltPositionSpace[0]
        self.possibilities['Focus'] = self.imaging.GetOptions(self.reqVS()).Focus
        self.test()
        
    # Wrapper for requests
    def req(self, name = None):
        res = {}
        if name != None:
            res = self.ptz.create_type(name)
        res['ProfileToken'] = self.media_token
        return res
    
    # Wrapper for video source requests
    def reqVS(self, name = None):
        res = {}
        if name != None:
            res = self.imaging.create_type(name)
        res['VideoSourceToken'] = self.video_token
        return res

    ### MOVING
    def getPosition(self):
        position = self.ptz.GetStatus(self.req('GetStatus')).Position
        return {'p': position.PanTilt._x, 't': position.PanTilt._y, 'z': position.Zoom._x}
    
    def stop(self):
        self.moveC(0,0,0)
        req = self.req('Stop')
        req.PanTilt = True
        req.Zoom = True
        self.ptz.Stop(req)
    
    # Move continuously
    def moveC(self, p=0, t=0, z=0, timeout=None):
        req = self.req('ContinuousMove')
        req.Velocity.PanTilt._x, req.Velocity.PanTilt._y = p, t
        req.Velocity.Zoom._x = z
        req.Timeout = timeout
        self.ptz.ContinuousMove(req)
        if(timeout != None):
            time.sleep(timeout)
            self.stop()
        
    # Move continuously by timings
    def moveCT(self, c=[0.0, 0.0, 0.0], t=[0, 0, 0]):
        t_idx = sorted(range(len(t)), key=lambda k: t[k])

        req1 = self.req('ContinuousMove')
        # longest action
        req1.Velocity.PanTilt._x, req1.Velocity.PanTilt._y, req1.Velocity.Zoom._x = c
        # second action
        req2 = self.req('ContinuousMove')
        c[t_idx[0]] = 0
        req2.Velocity.PanTilt._x, req2.Velocity.PanTilt._y, req2.Velocity.Zoom._x = c
        # third action
        req3  = self.req('ContinuousMove')
        c[t_idx[1]] = 0
        req2.Velocity.PanTilt._x, req2.Velocity.PanTilt._y, req2.Velocity.Zoom._x = c

        self.ptz.ContinuousMove(req1)
        time.sleep(t[t_idx[0]]) # run first and wait
        self.ptz.ContinuousMove(req2)
        time.sleep(t[t_idx[1]]) 
        self.ptz.ContinuousMove(req3)
        time.sleep(t[t_idx[2]])
        self.stop()
    
    # Move absolutely
    def moveA (self, p=0, t=0, z=0):
        req = self.req('AbsoluteMove')
        req.Position.PanTilt._x, req.Position.PanTilt._y = p, t
        req.Position.Zoom._x = z
        self.ptz.AbsoluteMove(req)
    
    ### FOCUSING
    def getFocus(self):
        return self.imaging.GetStatus(self.reqVS()).FocusStatus20.Position
    
    def setFocusMode(self, mode='MANUAL'):
        req = self.reqVS('SetImagingSettings')
        curr = self.imaging.GetImagingSettings(self.reqVS())
        req.ImagingSettings = curr.ImagingSettings
        req.ImagingSettings.Focus.AutoFocusMode = mode
        self.imaging.SetImagingSettings(req)
        
    # Move focus absolutely
    def moveFocusA(self, to):
        req = self.reqVS('Move')
        req.Focus.Absolute.Position = to
        req.Focus.Absolute.Speed = 1
        self.imaging.Move(req)
    
    # Move focus relatively
    def moveFocusR(self, to):
        req = self.reqVS('Move')
        req.Focus.Relative.Distance = to
        self.imaging.Move(req)
    
    # Move focus continuously
    def moveFocusC(self, speed):
        req = self.reqVS('Move')
        req.Focus.Continuous.Speed = speed
        self.imaging.Move(req)
        
    # Stop focusing
    def stopFocus(self):
        self.moveFocusC(0)
        self.imaging.Stop(self.reqVS('Stop'))
    
    ### TESTS
    def test(self):
        print 'Run tests'
        
        position = self.getPosition()
        print "P:" + str(position['p']) + " T:" + str(position['t']) + " Z:" + str(position['z'])
        
        # ---
        # test abs move
        # ---
        print "---"
        print "Test absolute move:"
        pos = self.getPosition()
        if(pos['p'] != self.possibilities['Absolute'].XRange.Min):
            print "Try move pan to " + str(self.possibilities['Absolute'].XRange.Min)
            self.moveA(self.possibilities['Absolute'].XRange.Min, pos['t'], pos['z'])
        else:
            self.moveA(self.possibilities['Absolute'].XRange.Max, pos['t'], pos['z'])
        time.sleep(2)
        if(pos['p'] != self.getPosition()['p']):
            self.possibilities['AbsoluteMove'] = True
        else:
            self.possibilities['AbsoluteMove'] = False
        self.stop()
        
        if(self.possibilities['AbsoluteMove']):
            print "Absolute move available"
        else:
            print "Absolute move unavailable"
            
        # ---
        # test focusing
        # ---
        print "---"
        print "Test focusing"
        
        try:
            focus = self.getFocus()
            print "Current focus:" + str(focus) # ALWAYS == 0 on 42
            self.possibilities['FocusGet'] = True
        except Exception:
            self.possibilities['FocusGet'] = False
            print "Getting focus unavailable"
            
        print "-"
        try:
            print "Set focus near"
            self.moveFocusA(self.possibilities['Focus'].NearLimit.Min)
            time.sleep(10)
            print "Set focus far"
            self.moveFocusA(self.possibilities['Focus'].NearLimit.Max)
            self.possibilities['FocusSetAbs'] = True
        except Exception:
            self.possibilities['FocusSetAbs'] = False
            print "Moving absolut focus is unavailable"
            
            
        print "-"
        try:
            print "Increase focus"
            self.moveFocusR(0.1)
            time.sleep(10)
            print "Decrease focus"
            self.moveFocusR(-0.1)
            self.possibilities['FocusSetRel'] = True
        except Exception:
            self.possibilities['FocusSetRel'] = False
            print "Moving relative focus is unavailable"
           
            
        print "-"
        try:
            print "Increase focus continuously"
            self.moveFocusC(1)
            time.sleep(5)
            print "Decrease focus continuously"
            self.moveFocusC(-1)
            time.sleep(5)
            print "Stop focus movement"
            self.stopFocus()
            self.possibilities['FocusSetRel'] = True
        except Exception:
            self.possibilities['FocusSetRel'] = False
            print "Moving focus continuously is unavailable"
            
                

In [57]:
cam = Camera('192.168.15.42', 80, 'ilyahandsup', 'xp3BJtSrR1IZ')

Run tests
P:-1.0 T:-0.5 Z:1.0
---
Test absolute move:
Absolute move available
---
Test focusing
Current focus:0.0
-
Set focus near
Moving absolut focus is unavailable
-
Increase focus
Moving relative focus is unavailable
-
Increase focus continuously
Decrease focus continuously
Stop focus movement


In [173]:
# cam.moveA(1,1,0)
# cam.moveC(1, -1, -1, 3)
# cam.moveCT([1,1,-1], [1,2,3])
# cam.stop()

In [51]:
cam.moveFocusC(-1)